In [5]:

# Let's say we have power outages (transformer blows out, driver hits a powerline and takes it down, lightning strike causes a tree to fall, etc.).
# We have to roll trucks to the site of the outage, so workers can restore power. We want to send the truck that is closest to the outage, rather
# than simply picking a random truck to go to the site. We have to make sure the guys in the truck have the proper skills/traiing to restore
# power, rather than just sedning two guys who know some things about powwer restoration, but not everything. Also, ideally we want to send a 
# team who just started working an hour ago, or a couple hours ago, but not 7-8 hours ago, because if we send the 7-8 hours team, we have to pay 
# them overtime, after they hit 8 hours for the shift. How can we do all of this? We can use optimization!!

# The details described here have elements reminiscent of a Traveling Saleserson Problem, but rather than optimizing
# (minimizing) the path of the salesperson this problem is a little more complex because we have to do the following:
# 1) Assigning Trucks to Outages
# 2) Skills Matching to Needs
# 3) Working Hours Constraint


In [15]:

# The optimization in this example is performed by a simple greedy algorithm 

import numpy as np

# Generate sample data
np.random.seed(42)
num_trucks = 5
num_outages = 10

truck_coordinates = np.random.rand(num_trucks, 2) * 100  # Random coordinates
outage_coordinates = np.random.rand(num_outages, 2) * 100  # Random coordinates

# Use labels for skills
skill_labels = ["Electrical", "Mechanical", "Communication"]
worker_skills = np.random.choice(skill_labels, size=(num_trucks, 3))

# Generate random hours worked by each worker
worker_hours_worked = np.random.randint(4, 8, size=num_trucks)

# Create a distance matrix
distance_matrix = np.linalg.norm(truck_coordinates[:, np.newaxis, :] - outage_coordinates, axis=2)


# Baseline assignment scenario (nearest available truck in a sequential manner)
baseline_assignment = []
assigned_trucks = set()

for outage_index in range(num_outages):
    min_distance = float('inf')
    assigned_truck = None
    for truck_index in range(num_trucks):
        if truck_index not in assigned_trucks:
            distance = distance_matrix[truck_index, outage_index]
            if distance < min_distance:
                min_distance = distance
                assigned_truck = truck_index
    baseline_assignment.append(assigned_truck)
    assigned_trucks.add(assigned_truck)
    # If all trucks are assigned, reset the assigned trucks set
    if len(assigned_trucks) == num_trucks:
        assigned_trucks = set()


# Optimized assignment scenario (closest truck to each outage)
optimized_assignment = np.argmin(distance_matrix, axis=0)

# Print baseline assignment scenario
print("\nBaseline Scenario:")
for outage_index, truck_index in enumerate(baseline_assignment):
    distance = distance_matrix[truck_index, outage_index]
    skills = ', '.join(worker_skills[truck_index])
    hours_worked = worker_hours_worked[truck_index]
    
    if hours_worked < 8:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked}")
    else:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked} (Overtime)")


# Print optimized assignment scenario
print("\nOptimized Scenario:")
for outage_index, truck_index in enumerate(optimized_assignment):
    distance = distance_matrix[truck_index, outage_index]
    skills = ', '.join(worker_skills[truck_index])
    hours_worked = worker_hours_worked[truck_index]
    
    if hours_worked < 8:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked}")
    else:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked} (Overtime)")


# Calculate and print overall savings
optimized_distance = distance_matrix[optimized_assignment, range(num_outages)].sum()
baseline_distance = distance_matrix[baseline_assignment, range(num_outages)].sum()

savings_percentage = ((baseline_distance - optimized_distance) / baseline_distance) * 100

print(f"\nOverall Savings: {savings_percentage:.2f}%")



Baseline Scenario:
Outage 1: Truck 4 (Mechanical, Electrical, Mechanical), Distance: 11.03 miles away, Hours Worked: 4
Outage 2: Truck 2 (Electrical, Communication, Mechanical), Distance: 39.92 miles away, Hours Worked: 6
Outage 3: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 3.76 miles away, Hours Worked: 7
Outage 4: Truck 5 (Electrical, Mechanical, Communication), Distance: 34.89 miles away, Hours Worked: 7
Outage 5: Truck 1 (Communication, Communication, Electrical), Distance: 66.20 miles away, Hours Worked: 7
Outage 6: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 45.61 miles away, Hours Worked: 7
Outage 7: Truck 5 (Electrical, Mechanical, Communication), Distance: 46.07 miles away, Hours Worked: 7
Outage 8: Truck 1 (Communication, Communication, Electrical), Distance: 18.45 miles away, Hours Worked: 7
Outage 9: Truck 4 (Mechanical, Electrical, Mechanical), Distance: 37.94 miles away, Hours Worked: 4
Outage 10: Truck 2 (Electrical, Communication, Mechanical), 

In [6]:

# In the example above, we used a straightforward, closest-distance heuristic

# For each outage, it assigns the closest truck by directly selecting the index of the minimum distance 
# from the distance_matrix using np.argmin.

# In the example below, we do the following:
# Create a Gurobi model.
# Define decision variables representing which truck is assigned to which outage.

# Set the Objective:
# The objective is to minimize the total distance traveled by the trucks.

# Add Constraints:
# Each outage should be assigned to exactly one truck.
# Each truck should be assigned to at most one outage.


In [16]:

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Generate sample data
np.random.seed(42)
num_trucks = 5
num_outages = 10

truck_coordinates = np.random.rand(num_trucks, 2) * 100  # Random coordinates
outage_coordinates = np.random.rand(num_outages, 2) * 100  # Random coordinates

# Create a distance matrix
distance_matrix = np.linalg.norm(truck_coordinates[:, np.newaxis, :] - outage_coordinates, axis=2)

# Create a Gurobi model
model = gp.Model("truck_assignment")

# Add decision variables
assignment = model.addVars(num_trucks, num_outages, vtype=GRB.BINARY, name="assignment")

# Objective: Minimize total distance
model.setObjective(gp.quicksum(distance_matrix[i, j] * assignment[i, j] for i in range(num_trucks) for j in range(num_outages)), GRB.MINIMIZE)

# Constraints: Each outage must be assigned to exactly one truck
model.addConstrs((gp.quicksum(assignment[i, j] for i in range(num_trucks)) == 1 for j in range(num_outages)), "Outage_Assignment")

# Constraints: Each truck can be assigned to at most 3 outages
model.addConstrs((gp.quicksum(assignment[i, j] for j in range(num_outages)) <= 3 for i in range(num_trucks)), "Truck_Assignment")

# Optimize the model
model.optimize()

# Check for infeasibility
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible; computing IIS...")
    model.computeIIS()
    model.write("infeasible.ilp")
    print("IIS written to infeasible.ilp")
else:
    # Extract and print the optimal solution
    optimized_assignment_gurobi = np.zeros(num_outages, dtype=int)
    for v in model.getVars():
        if v.x > 0.5:  # Variable is binary, so this is essentially checking if it's 1
            truck, outage = map(int, v.varName.split('[')[1].strip(']').split(','))
            optimized_assignment_gurobi[outage] = truck

    # Print optimized assignment scenario
    print("\nOptimized Scenario with Gurobi:")
    for outage_index, truck_index in enumerate(optimized_assignment_gurobi):
        distance = distance_matrix[truck_index, outage_index]
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1}, Distance: {distance:.2f} miles")

    # Calculate and print overall savings
    optimized_distance = distance_matrix[optimized_assignment_gurobi, range(num_outages)].sum()
    baseline_distance = distance_matrix[baseline_assignment, range(num_outages)].sum()

    savings_percentage = ((baseline_distance - optimized_distance) / baseline_distance) * 100

    print(f"\nOverall Savings: {savings_percentage:.2f}%")
    

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 15 rows, 50 columns and 100 nonzeros
Model fingerprint: 0x1948502a
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 524.4393338
Presolve time: 0.00s
Presolved: 15 rows, 50 columns, 100 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 318.6071192

Root relaxation: objective 3.041697e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [17]:

# We can further optimize the costs and benefits of optimizing this part of the business by sending multiple 
# trucks to an outage, we can extend the current optimization model. The key is to balance the cost of 
# deploying more trucks (more expensive for the enterprise) with the benefit of faster outage resolution
# (power is restored quicker). It's almost certainly better to restore power quicker rather than slower, simply
# for the benefit of the consumer, because it sucks to be without power, and if your response time is too slow, 
# you will be on the front page of lots of major newspapers, and NOT in a good way!! 
 

In [18]:

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Generate sample data
np.random.seed(42)
num_trucks = 5
num_outages = 10

truck_coordinates = np.random.rand(num_trucks, 2) * 100  # Random coordinates
outage_coordinates = np.random.rand(num_outages, 2) * 100  # Random coordinates

# Create a distance matrix
distance_matrix = np.linalg.norm(truck_coordinates[:, np.newaxis, :] - outage_coordinates, axis=2)

# Define costs
truck_cost_per_hour = 50  # Cost per hour for each truck
penalty_cost_per_hour = 200  # Penalty cost per hour for each outage
base_resolution_time = 8  # Base time to resolve an outage with 1 truck (in hours)
diminishing_factor = 0.5  # Factor by which additional trucks reduce resolution time

# Create a Gurobi model
model = gp.Model("truck_assignment")

# Add decision variables
assignment = model.addVars(num_trucks, num_outages, vtype=GRB.BINARY, name="assignment")

# Auxiliary variables for piecewise linear approximation of diminishing returns
outage_resolution_time = model.addVars(num_outages, vtype=GRB.CONTINUOUS, name="outage_resolution_time")
slack_variables = model.addVars(num_outages, vtype=GRB.CONTINUOUS, name="slack")

# Objective: Minimize total cost
total_cost = gp.quicksum(distance_matrix[i, j] * assignment[i, j] * truck_cost_per_hour for i in range(num_trucks) for j in range(num_outages))
total_cost += gp.quicksum(penalty_cost_per_hour * (outage_resolution_time[j] + slack_variables[j]) for j in range(num_outages))

model.setObjective(total_cost, GRB.MINIMIZE)

# Constraints: Each outage must be assigned at least one truck
model.addConstrs((gp.quicksum(assignment[i, j] for i in range(num_trucks)) >= 1 for j in range(num_outages)), "Outage_Assignment")

# Constraints: Each truck can be assigned to at most 3 outages
model.addConstrs((gp.quicksum(assignment[i, j] for j in range(num_outages)) <= 3 for i in range(num_trucks)), "Truck_Assignment")

# Piecewise linear approximation for diminishing returns on resolution time
for j in range(num_outages):
    num_trucks_assigned = gp.quicksum(assignment[i, j] for i in range(num_trucks))
    model.addConstr(outage_resolution_time[j] == base_resolution_time - diminishing_factor * num_trucks_assigned + slack_variables[j], f"Resolution_Time_{j}")

# Optimize the model
model.optimize()

# Check for infeasibility
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible; computing IIS...")
    model.computeIIS()
    model.write("infeasible.ilp")
    print("IIS written to infeasible.ilp")
else:
    # Extract and print the optimal solution
    optimized_assignment_gurobi = {i: [] for i in range(num_trucks)}
    for v in model.getVars():
        if 'assignment' in v.varName and v.x > 0.5:  # Variable is binary, so this is essentially checking if it's 1
            truck, outage = map(int, v.varName.split('[')[1].strip(']').split(','))
            optimized_assignment_gurobi[truck].append(outage)

    # Print optimized assignment scenario
    print("\nOptimized Scenario with Gurobi:")
    for truck, outages in optimized_assignment_gurobi.items():
        for outage in outages:
            distance = distance_matrix[truck, outage]
            print(f"Truck {truck + 1}: Outage {outage + 1}, Distance: {distance:.2f} miles")

    # Calculate and print overall cost
    optimized_cost = model.objVal
    print(f"\nOptimized Total Cost: ${optimized_cost:.2f}")

# Analysis of adding more trucks
print("\nAnalysis of Adding More Trucks:")
for additional_trucks in range(1, 4):
    model.reset()
    
    # Add constraints to allow more trucks per outage
    model.addConstrs((gp.quicksum(assignment[i, j] for i in range(num_trucks)) == additional_trucks for j in range(num_outages)), f"Additional_Trucks_{additional_trucks}")
    
    model.optimize()
    
    if model.status == GRB.INFEASIBLE:
        print(f"Model with {additional_trucks + 1} trucks per outage is infeasible.")
    else:
        optimized_cost = model.objVal
        print(f"\nOptimized Total Cost with {additional_trucks + 1} trucks per outage: ${optimized_cost:.2f}")
        

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 25 rows, 70 columns and 170 nonzeros
Model fingerprint: 0x9e47da1f
Variable types: 20 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 41221.966688
Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolved: 15 rows, 50 columns, 100 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 30930.355962

Root relaxation: objective 3.020848e+04, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [19]:

# Current Feasible Solution:
# With up to 2 trucks per outage, the optimization model finds a feasible and optimal solution.
# The optimized total cost is $30,208.48.

# Infeasibility Beyond 2 Trucks:
# The model becomes infeasible when attempting to assign 3 or more trucks per outage.
# This infeasibility might be due to constraints that are too restrictive or conflicts arising from the number of available trucks and 
# their assignments.


In [ ]:

# END!!! 
